In [ ]:
from google.colab import userdata

CHAT_ID = 8391552054
telegram_token = userdata.get('telegram_token')


In [ ]:
!pip install python-telegram-bot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.0/731.0 kB 36.5 MB/s eta 0:00:00


In [ ]:
from telegram import Bot
import asyncio

text = "Hello from Python"
async def send_message():
    bot = Bot(token = telegram_token)
    await bot.send_message(chat_id=CHAT_ID, text=text)

await send_message()

In [10]:
!pip install Mastodon.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 8.7 MB/s eta 0:00:00


<frozen posixpath>:82: RuntimeWarning: coroutine 'send_message' was never awaited


In [12]:
dummy_post = "This is a dummy post"

In [15]:
# ===== COPY / PASTE: Telegram HITL (Approve/Reject) -> Mastodon post (Colab/Jupyter safe) =====
# Works with python-telegram-bot v20+ and Mastodon.py
# IMPORTANT: Replace the 3 placeholders: BOT_TOKEN, MASTODON_BASE_URL, MASTODON_ACCESS_TOKEN

!pip -q install -U python-telegram-bot Mastodon.py

import asyncio
import logging

from telegram import InlineKeyboardButton, InlineKeyboardMarkup, Update
from telegram.ext import (
    Application,
    CommandHandler,
    CallbackQueryHandler,
    ContextTypes,
)

from mastodon import Mastodon

# ---------------- CONFIG ----------------
BOT_TOKEN = userdata.get('telegram_token')
MASTODON_BASE_URL = "https://mastodon.social"  # e.g. "https://mastodon.social" or your instance
MASTODON_ACCESS_TOKEN = userdata.get('MASTODON_API')

# You said you've already defined dummy_post; keeping a fallback here just in case.
try:
    dummy_post
except NameError:
    dummy_post = "Hello from Colab HITL. (fallback dummy_post)"

# -------------- LOGGING -----------------
logging.basicConfig(level=logging.INFO)

# ------------- MASTODON CLIENT ----------
mastodon = Mastodon(
    access_token=MASTODON_ACCESS_TOKEN,
    api_base_url=MASTODON_BASE_URL,
)

async def post_to_mastodon(text: str):
    # Mastodon.py is sync; run it in a worker thread so we don't block the async loop.
    return await asyncio.to_thread(mastodon.status_post, text)

# ------------- TELEGRAM HANDLERS --------
async def test(update: Update, context: ContextTypes.DEFAULT_TYPE):
    keyboard = InlineKeyboardMarkup([
        [
            InlineKeyboardButton("Approve", callback_data="approve"),
            InlineKeyboardButton("Reject", callback_data="reject"),
        ]
    ])
    await update.message.reply_text(
        "Approve posting `dummy_post` to Mastodon?",
        reply_markup=keyboard,
    )

async def handle_callback(update: Update, context: ContextTypes.DEFAULT_TYPE):
    query = update.callback_query
    await query.answer()

    if query.data == "approve":
        try:
            status = await post_to_mastodon(dummy_post)
            # status is a dict-like object; "url" is commonly present
            url = status.get("url") if isinstance(status, dict) else getattr(status, "url", None)
            if url:
                await query.edit_message_text(f"APPROVED ✅\nPosted to Mastodon:\n{url}")
            else:
                await query.edit_message_text("APPROVED ✅\nPosted to Mastodon.")
        except Exception as e:
            logging.exception("Mastodon post failed")
            await query.edit_message_text(f"APPROVED ✅\nBut Mastodon post failed:\n{type(e).__name__}: {e}")
    else:
        await query.edit_message_text("REJECTED ❌")

# ------------- BUILD APP -----------------
app = Application.builder().token(BOT_TOKEN).build()
app.add_handler(CommandHandler("test", test))
app.add_handler(CallbackQueryHandler(handle_callback))

# ------------- NOTEBOOK-SAFE START/STOP ---------------
# In Colab/Jupyter, DO NOT use app.run_polling() (it tries to own the event loop).
# Use this async start routine instead.

# If you re-run this cell, attempt to stop any prior polling cleanly first.
async def start_bot():
    # best-effort cleanup if already running
    try:
        if getattr(app, "running", False):
            await app.updater.stop()
            await app.stop()
            await app.shutdown()
    except Exception:
        pass

    await app.initialize()
    await app.start()
    await app.updater.start_polling()
    print("✅ Bot is polling. In Telegram, send: /test")
    print("To stop later, run: await stop_bot()")

async def stop_bot():
    await app.updater.stop()
    await app.stop()
    await app.shutdown()
    print("🛑 Bot stopped.")

# Start it
await start_bot()


✅ Bot is polling. In Telegram, send: /test
To stop later, run: await stop_bot()


In [16]:
await stop_bot()

🛑 Bot stopped.
